# Tables vs. views

Views have been described as "virtual tables". It's true that views are similar to tables in certain aspects, but there are key differences. 

<img src="images/03.02.jpg" style="width:800px;height:300px;">

# Viewing views

Because views are very useful, it's common to end up with many of them in your database. It's important to keep track of them so that database users know what is available to them.

```
-- Get all non-systems views
SELECT * FROM INFORMATION_SCHEMA.views
WHERE table_schema NOT IN ('pg_catalog', 'information_schema');
```

### What does it do?

```
SELECT content.reviewid,
    content.content
   FROM content
  WHERE (length(content.content) > 4000);
```

- Returns the content records that have reviews of more than 4000 characters.

### What does it do?

```
 SELECT reviews.reviewid,
    reviews.title,
    reviews.score
   FROM reviews
  WHERE (reviews.pub_year = 2017)
  ORDER BY reviews.score DESC
 LIMIT 10;
```

- Returns the top 10 highest scored reviews published in 2017.

# Creating and querying a view

In these Pitchfork reviews, we're particularly interested in high-scoring reviews and if there's a common thread between the works that get high scores. You'll make a view to help with this analysis so that we don't have to type out the same query often to get these high-scoring reviews.

```
-- Create a view for reviews with a score above 9
CREATE VIEW high_scores AS
SELECT * FROM REVIEWS
WHERE score > 9;

-- Count the number of self-released works in high_scores
SELECT COUNT(*) FROM high_scores
INNER JOIN labels ON high_scores.reviewid = labels.reviewid
WHERE label = 'self-released';
```

# Creating a view from other views

Views can be created from queries that include other views. This is useful when you have a complex schema, potentially due to normalization, because it helps reduce the JOINS needed. The biggest concern is keeping track of dependencies, specifically how any modifying or dropping of a view may affect other views.

```
-- Create a view with the top artists in 2017
CREATE VIEW top_artists_2017 AS
-- with only one column holding the artist field
SELECT artist_title.artist FROM artist_title
INNER JOIN top_15_2017
ON artist_title.reviewid = top_15_2017.reviewid;

-- Output the new view
SELECT * FROM top_artists_2017;
```

### Which is the `DROP` command that would drop both `top_15_2017` and `top_artists_2017`?
- `DROP VIEW top_15_2017 CASCADE;`

# Granting and revoking access

Access control is a key aspect of database management. Not all database users have the same needs and goals, from analysts, clerks, data scientists, to data engineers. As a general rule of thumb, write access should never be the default and only be given when necessary.

```
-- Revoke everyone's update and insert privileges
REVOKE INSERT, UPDATE ON long_reviews FROM PUBLIC; 

-- Grant the editor update and insert privileges 
GRANT INSERT, UPDATE ON long_reviews TO editor; 
```

# Updatable views

we've used the `information_schema.views` to get all the views in a database. If you take a closer look at this table, you will notice a column that indicates whether the view is updatable.

```
SELECT * FROM information_schema.views 
WHERE
is_updatable = 'YES'
```

# Redefining a view

Unlike inserting and updating, redefining a view doesn't mean modifying the actual data a view holds. Rather, it means modifying the underlying query that makes the view. 

The artist_title view needs to be appended to include a column for the label field from the labels table. Can the CREATE OR REPLACE statement be used to redefine the artist_title view?
- Yes, as long as the label column comes at the end.

```
-- Redefine the artist_title view to have a label column
CREATE OR REPLACE VIEW artist_title AS
SELECT reviews.reviewid, reviews.title, artists.artist, labels.label
FROM reviews
INNER JOIN artists
ON reviews.reviewid = artists.reviewid
INNER JOIN labels
ON reviews.reviewid = labels.reviewid;

SELECT * FROM artist_title;
```

# Materialized versus non-materialized

Materialized and non-materialized are two distinct categories of views.

<img src="images/03.11.jpg" style="width:800px;height:300px;">


# Creating and refreshing a materialized view

The syntax for creating materialized and non-materialized views are quite similar because they are both defined by a query. One key difference is that we can refresh materialized views, while no such concept exists for non-materialized views. It's important to know how to refresh a materialized view, otherwise the view will remain a snapshot of the time the view was created.

```
-- Create a materialized view called genre_count 
CREATE MATERIALIZED VIEW genre_count AS
SELECT genre, COUNT(*) 
FROM genres
GROUP BY genre;

INSERT INTO genres
VALUES (50000, 'classical');

-- Refresh genre_count
REFRESH MATERIALIZED VIEW genre_count;

SELECT * FROM genre_count;
```

# Managing materialized views

Why do companies use pipeline schedulers, such as Airflow and Luigi, to manage materialized views?
- To refresh materialized views with consideration to dependences between views. These pipeline schedulers help visualize dependencies and create a logical order for refreshing views.